# Manoeuvring simulation in seaman

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import quantities as pq
import matplotlib.pyplot as plt
import os

import evaluation

# Seaman:
import seaman.tests
from seaman.systems.cppsystems.world import World
from seaman.systems.composite.dynamic_ship import DynamicShip
from seaman.simulations.manoeuvring import TurningCircle,ZigZag,SelfPropulsion


## Create a world and load a ship

In [ ]:
w = World()
ship = DynamicShip.from_shipfile(path=seaman.tests.test_ship_path)
w.register_unit(ship)

## Run a turning circle simulation

In [ ]:
simulation = TurningCircle(ship = ship)
simulation.dt = 0.1
#simulation.dt = 2
simulation.max_simulation_time = 1000

### Set some parameters for this turning circle:
You need to specify the physical unit using the Quantities package. 
This way we avoid to know in which physical unit parameters should be specified. 
[knots] will be automatically converted into [m/s], [degrees] into [radians] etc.

In [ ]:
delta = pq.Quantity(35, pq.deg)
u = 12 * 1.852 / 3.6
rev = pq.Quantity(3.23568412333035, 1 / pq.s)
simulation.setup_simulation_parameters(speed = u,angle = delta,rev = rev, pullout=False)

Now run the simulation. The *TurningCircle* class knows how this simulation is performed in the correct way. 

In [ ]:
simulation.ship.cog.bl.rudder_0.store_all()
simulation.run()

## Save simulation
It is possible to save the simulation results into a HDF5 file:

In [ ]:
save_directory = os.path.abspath('results')
if not os.path.exists(save_directory):
    os.mkdir(save_directory)

save_path = os.path.join(save_directory,'test.hdf5')

simulation.save_current_simulation(path = save_path)

## Post-process the simulation

### Directly
You can process the data directly by accessing the results that simulation object holds in memory.  

In [ ]:
positions = simulation.ship.res.position_world
fig,ax=plt.subplots()
x0 = positions[:,0]
y0 = positions[:,1]

ax.plot(y0,x0)
ax.set_aspect('equal', 'box')

fig,ax=plt.subplots()
time = simulation.ship.res.time  
psi = simulation.ship.res.attitude_world[:,2]
ax.plot(time,psi)


### Using Evaluation
You can process the data in a more standard way by using the *Evaluation* package. 
*Evaluation* can analyze time series data from all SSPA facilities (Towingtank and MDL) but is 
also compatable with Seaman. There are benifits with having just ONE code to deal with data regardless of 
its origin. So that a ZigZag simulation in Seaman is treated the same way as a ZigZag model tests in MDL 
etc.

In [ ]:
from evaluation.run_manoeuvring import RunZigZag, RunTurningCircle
from evaluation.run import Run
from evaluation.run_dynamic import RunDynamic


In [ ]:
 interesting_datasets = [r"ship/position_world/x",
                            r"ship/position_world/y",
                            r"ship/position_world/z",
                            r"ship/attitude_world/x",
                            r"ship/attitude_world/y",
                            r"ship/attitude_world/z",
                            r"ship/cog/bl/rudder_0/delta",
                            ]

rename_channels = {
    r"ship/cog/bl/rudder_0/delta":'delta',
}

hdf5_file_path = save_path
run_name = 'seaman run'

meta_data = {
    'TrackSpeed':u,
    'Beam':simulation.ship.static_ship.shipdict.main_data['beam'],
    'Lpp':simulation.ship.static_ship.shipdict.main_data['l'],
    'ScaleFactor':1,

    }

units = {
    'TrackSpeed':'m/s',
    'Beam':'m',
    'Lpp':'m',
    'ScaleFactor':36,

    }

run = RunTurningCircle.load_time_series_from_seaman(run_name=run_name,hdf5_file_path=hdf5_file_path,
                                                    interesting_datasets=interesting_datasets,
                                                    meta_data=meta_data, rename_channels=rename_channels, 
                                                    units=units)

### Evaluate
Now we can evaluate the results and look at the data

In [ ]:
run.evaluate()
results = run.results['turning_circle']
units = results.pop('units')
results

In [ ]:
run.track_plot()



In [ ]:
run.plot_summary();


In [ ]:
run.plot('V')
run.plot('beta')
